# DEXES memecoins

## Importaciones

In [1]:
from solders.keypair import Keypair
from typing import Literal, Union

from solana_manager import SolanaAccountInfo, SolanaWalletManager
from pumpfun import PumpFunTransactions, PumpFunPriceFetcher
from dexscreener import DexScreenerPriceTracker, DexScreenerPortfolioMonitor

## Configuraciones

In [2]:
NETWORK = "mainnet-beta"
RPC_URL = "https://api.mainnet-beta.solana.com"
WALLET_FILE = "wallets/wallet_pumpfun.json"
SLIPPAGE = 15.0
PRIORITY_FEE = 0.0001

wallet_data = None
async with SolanaWalletManager(network=NETWORK) as solana_manager:
    wallet_data = await solana_manager.load_wallet_from_file(filename=WALLET_FILE)
    wallet_data["key_pair"] = Keypair().from_base58_string(wallet_data["private_key"])

account_info = SolanaAccountInfo(network=NETWORK)
transactions = PumpFunTransactions()
pump_fetcher = PumpFunPriceFetcher()
price_tracker = DexScreenerPriceTracker()

if not wallet_data:
    raise Exception("Wallet not found")

portfolio_monitor = DexScreenerPortfolioMonitor(wallet_data.get("public_key"), price_tracker)

🌐 Conectado a Solana mainnet-beta (RPC: https://api.mainnet-beta.solana.com)
📂 Wallet cargada desde wallets/wallet_pumpfun.json
🔌 Conexión a Solana cerrada.
🌐 PumpFun API Client inicializado
   WebSocket: ❌ wss://pumpportal.fun/api/data
   HTTP: ✅ https://pumpportal.fun/api
   API Key: ❌ No proporcionada
🎯 Pump.fun Price Fetcher inicializado
🌐 Configurado para conectar a https://api.mainnet-beta.solana.com
📊 DexScreener Price Tracker inicializado (Async)
📊 DexScreener Portfolio Monitor inicializado (Async)
📍 Wallet: GMN2f6Ps...dfBtnAtz


📂 No se encontraron datos previos del portfolio


## Información

### Balance

In [3]:
async with portfolio_monitor:
    await portfolio_monitor.get_detailed_balance()


💰 BALANCE DETALLADO DE WALLET
🪙 SOL:
   Balance: 0.211584 SOL
   Precio: $178.56 USD
   Valor: $37.78 USD

🪙 TOKENS (5 encontrados):
💰 Obteniendo precio para: 6F6jjd71...
⚠️ Error en endpoint tokens: 'NoneType' object has no attribute 'get'
⚠️ Error en endpoint pairs: 'NoneType' object has no attribute 'get'
❌ No se pudo obtener precio para 6F6jjd71... en ninguna fuente

📊 RESUMEN TOTAL:
   💰 SOL: 0.211584 SOL ($37.78 USD)
   🪙 Tokens: $0.000000 USD
   💎 TOTAL USD: $37.78
   💎 TOTAL SOL: 0.211584 SOL
🔒 DexScreener Portfolio Monitor cerrado


# Trading

In [4]:
TOKEN_MINT="6F6jjd71nbjwUZNi93wpXimbyH12CLkeMNJ6X1brpump"

### Precio de token

In [5]:
token_price = None
async with price_tracker:
    token_price = await price_tracker.get_token_price(TOKEN_MINT)

if not token_price:
    print("🔄 Reintentando con Pump.fun Price Fetcher")
    token_price = None
    async with pump_fetcher:
        token_price = await pump_fetcher.get_token_price(TOKEN_MINT)

if token_price:
    print(f"\n🎉 PRECIO OBTENIDO EXITOSAMENTE!")
    print(f"💰 Precio SOL: {token_price.price_sol:.12f}")
    print(f"💵 Precio USD: ${token_price.price_usd:.12f}")
    if hasattr(token_price, 'market_cap_usd'):
        print(f"📊 Market Cap: ${token_price.market_cap_usd:,.2f}")
    if hasattr(token_price, 'bonding_progress'):
        print(f"📈 Progreso Bonding: {token_price.bonding_progress:.2f}%")
    print(f"⏰ Timestamp: {token_price.timestamp}")
else:
    print("❌ No se pudo obtener el precio")

💰 Obteniendo precio para: 6F6jjd71...
✅ Precio obtenido desde DexScreener tokens: $0.0000049740 USD
🔒 DexScreener Price Tracker cerrado

🎉 PRECIO OBTENIDO EXITOSAMENTE!
💰 Precio SOL: 0.000000027856
💵 Precio USD: $0.000004974000
⏰ Timestamp: 2025-07-30 18:32:13.664531


### Local trade

In [47]:
ACTION: Literal["buy", "sell"] = "buy"
AMOUNT: Union[float, str] = "0.000001"
DENOMINATED_IN_SOL: bool = True
SLIPPAGE = "15.0"
PRIORITY_FEE = "0.000001"

async with transactions:
    signature = await transactions.create_and_send_local_trade(
        keypair=wallet_data.get("key_pair"), # type: ignore
        action=ACTION,
        mint=TOKEN_MINT,
        amount=AMOUNT,
        denominated_in_sol=DENOMINATED_IN_SOL,
        slippage=float(SLIPPAGE),
        priority_fee=float(PRIORITY_FEE)
    )
    print(f"🔗 Transacción enviada exitosamente. Puedes verla en Solscan: https://solscan.io/tx/{signature}?cluster=mainnet-beta")

🔌 Iniciando sesión de transacciones PumpFun...
✅ Sesión HTTP inicializada
🛠️ Creando transacción local: buy 0.000001 de 6F6jjd71nbjwUZNi93wpXimbyH12CLkeMNJ6X1brpump
📄 Transacción recibida, firmando localmente...
🖋️ Transacción firmada, enviando a https://api.mainnet-beta.solana.com/...
✅ Transacción enviada. Firma: 3Fy27ESeW4A6tMJ8mrqxt9HcZ8hnnmoiN2JNyqZKmLhstN6K91y6a2yPXqwTmNFq4bhBtaiKuDMqaWzPjgm7oe1B
🔗 Transacción enviada exitosamente. Puedes verla en Solscan: https://solscan.io/tx/3Fy27ESeW4A6tMJ8mrqxt9HcZ8hnnmoiN2JNyqZKmLhstN6K91y6a2yPXqwTmNFq4bhBtaiKuDMqaWzPjgm7oe1B?cluster=mainnet-beta
🔌 Cerrando sesión de transacciones PumpFun...
🔌 Sesión HTTP cerrada
✅ Sesión de transacciones cerrada correctamente
